# Trained model evaluation

In [7]:
import os, sys

sys.path.append( os.path.abspath('../../Dataset/scripts'))
from utils import *

sys.path.append( os.path.abspath('../model') )
from train import *
import model as MODEL

In [47]:
""" Parameters """
# model_path = os.path.abspath('../notebook/train_log_20181020/model/backup/model.ckpt')
model_path = os.path.abspath('../notebook/train_log_20181020/model/model.ckpt')

data_test_path = os.path.join( \
                      os.path.abspath('../../Dataset/dataset/db_aug_v2_20181019_ndb_aug_v2_20181020'), 'test.h5' \
                             )

""" Parameters """
GPU_INDEX = 0
NUM_POINT = 256
NUM_CLASSES = 3

BN_INIT_DECAY = 0.5
BN_DECAY_DECAY_RATE = 0.5
BN_DECAY_DECAY_STEP = float(200000)
BN_DECAY_CLIP = 0.99
DECAY_STEP = BN_DECAY_DECAY_STEP
DECAY_RATE = BN_DECAY_DECAY_RATE

BATCH_SIZE = 10

MOMENTUM = 0.9

BASE_LEARNING_RATE = 0.001

OPTIMIZER = 'adam'

MAX_EPOCH = 20

In [48]:
""" Load dataset """
data, label = load_h5(data_test_path)


""" Data statistics"""
label_list = [0,1,2]

y_val = []
for j in range ( len ( label_list ) ):
    y_val.append(np.sum(label == label_list[j]))

x_name=('unknown-test', 'cars-test', 'trucks-test')

index = range( len(x_name) )

plt.figure(figsize=(15,5))
plt.bar(index, y_val, tick_label=x_name, align='center')
plt.ylabel('Number of dataset')
plt.title('Label distribution')
plt.xlim( -1, len(x_name))
plt.ylim( 0, np.max(y_val) * 1.1 )
plt.show()

In [49]:
is_training = False

# reset graph
tf.reset_default_graph()

with tf.device('/gpu:'+str(GPU_INDEX)):
    pointclouds_pl, labels_pl = MODEL.placeholder_inputs(BATCH_SIZE, NUM_POINT)
    is_training_pl = tf.placeholder(tf.bool, shape=())

    # simple model
    pred, end_points = MODEL.get_model(pointclouds_pl, is_training_pl)
    loss = MODEL.get_loss(pred, labels_pl, end_points)

    # Add ops to save and restore all the variables.
    saver = tf.train.Saver()

# Create a session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement = True
config.log_device_placement = True

ops = {'pointclouds_pl': pointclouds_pl,
       'labels_pl': labels_pl,
       'is_training_pl': is_training_pl,
       'pred': pred,
       'loss': loss}

# ops.reset_default_graph() 

sess = tf.Session(config=config)

# Restore variables from disk.
saver.restore(sess, model_path)

print "Model restored"
sys.stdout.flush()

## Evaluation using test set    
is_training = False
total_correct = 0
total_seen = 0
total_seen_class = [0 for _ in range(NUM_CLASSES)]
total_correct_class = [0 for _ in range(NUM_CLASSES)]
total_detect_class = [0 for _ in range(NUM_CLASSES)]
total_detect_true_class = [0 for _ in range(NUM_CLASSES)]

current_data = data
current_label = label

num_batches = current_data.shape[0] // BATCH_SIZE

results = []

for batch_idx in trange(num_batches):
    start_idx = batch_idx * BATCH_SIZE
    end_idx = (batch_idx+1) * BATCH_SIZE

    feed_dict = {ops['pointclouds_pl']: current_data[start_idx:end_idx],
                 ops['labels_pl']: current_label[start_idx:end_idx],
                 ops['is_training_pl']: is_training}
    loss_val, pred_val = sess.run([ops['loss'], ops['pred']],
                              feed_dict=feed_dict)
    
    pred_val = np.argmax(pred_val, 1)
    results.extend(pred_val)


INFO:tensorflow:Summary name classify loss is illegal; using classify_loss instead.
INFO:tensorflow:Summary name mat loss is illegal; using mat_loss instead.
INFO:tensorflow:Restoring parameters from /home/gozilla/kyungpyo/git/MasterThesisProject/Network/notebook/train_log_20181020/model/model.ckpt
Model restored


100%|██████████| 1461/1461 [00:23<00:00, 62.92it/s]


In [51]:
prediction = np.array(results)
ground_truth = np.array(current_label[:prediction.shape[0]])
correct = np.sum( prediction == ground_truth )

incorrect_idx = []

for i, gt in enumerate(ground_truth):
    
    pred = prediction[i]
    
    if not gt == pred:
        
        incorrect_idx.append(i)
        
        print "gt: {}, pred: {}".format(gt, pred)

print "Correct / incorrect : {} / {} ".format( ground_truth.shape[0] - len(incorrect_idx) , len(incorrect_idx) )

gt: 0, pred: 2
gt: 0, pred: 2
gt: 0, pred: 2
gt: 0, pred: 2
gt: 2, pred: 0
gt: 2, pred: 0
gt: 2, pred: 0
Correct / incorrect : 14603 / 7 


In [53]:
""" Display """

%matplotlib qt

fig = plt.figure(figsize=(15,5))
    
# display point cloud
for i, idx in enumerate(incorrect_idx):

    ax = fig.add_subplot(331 + i, projection='3d')
    ax = display_point_cloud_box_ax(ax, data[idx])
    ax.set_title("idx: {}, gt: {}, pred: {}".format(idx, ground_truth[idx], prediction[idx]))
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    
plt.show()
